# 176. Second Highest Salary
[Problem description](https://leetcode.com/problems/second-highest-salary/description/)

In [69]:
%%html
<style>
table {float:left}
</style>

In [70]:
import sys
sys.version

'3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]'

In [71]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]')\
    .appName('LeetCode-176-PySpark')\
    .getOrCreate()

In [72]:
# Data 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
employee_schema = StructType([
    StructField('id', IntegerType(), False),
    StructField('salary', IntegerType(), False),
])
employee = spark.createDataFrame([[1, 100], [2, 200], [3, 300]], schema=employee_schema)
employee.show()


+---+------+
| id|salary|
+---+------+
|  1|   100|
|  2|   200|
|  3|   300|
+---+------+



**Write a solution to find the second highest salary from the Employee table. If there is no second highest salary, return null.**

| SecondHighestSalary |
|---------------------|
| 200                 |

In [73]:
# Solution
from pyspark.sql import Window
from pyspark.sql.functions import lit, col, when, nth_value, desc, last_value
from pyspark.sql.dataframe import DataFrame

def second_highest_salary(employee: DataFrame):
    window = Window.orderBy(desc('salary'))
    df = employee.select('salary').distinct()\
        .withColumn('SecondHighestSalary', nth_value('salary', 2).over(window))\
        .select(last_value('SecondHighestSalary').alias('SecondHighestSalary'))
    return df

result = second_highest_salary(employee)
result.show()


+-------------------+
|SecondHighestSalary|
+-------------------+
|                200|
+-------------------+



In [74]:
test_data = [
    [[1, 100], [2, 200], [3, 300]],
    [[1, 100]],
    [[1, 100], [2, 100]],
    [[1, 100], [2, 200]],
]
test_result = ['200', 'null', 'null', '100',]

# Run tests
for idx, data in enumerate(zip(test_data, test_result)):
    test_input, exp_result = data
    print(f'{idx+1}. test:')
    print('input:', test_input)
    print('expected output:', exp_result)
    
    employee_data = spark.createDataFrame(test_input, schema=employee_schema)
    result = second_highest_salary(employee_data)
    result.show()
    print()

1. test:
input: [[1, 100], [2, 200], [3, 300]]
expected output: 200
+-------------------+
|SecondHighestSalary|
+-------------------+
|                200|
+-------------------+


2. test:
input: [[1, 100]]
expected output: null
+-------------------+
|SecondHighestSalary|
+-------------------+
|               NULL|
+-------------------+


3. test:
input: [[1, 100], [2, 100]]
expected output: null
+-------------------+
|SecondHighestSalary|
+-------------------+
|               NULL|
+-------------------+


4. test:
input: [[1, 100], [2, 200]]
expected output: 100
+-------------------+
|SecondHighestSalary|
+-------------------+
|                100|
+-------------------+


